# 📊 HTML Report System - Google Colab Edition

This notebook allows you to use the HTML Report System directly in Google Colab!

## Features:
- ✅ Run completely in the cloud
- ✅ No local installation needed
- ✅ Edit sections and save data
- ✅ Generate PDFs directly in Colab
- ✅ Download your work anytime

## Quick Start:
1. Run all cells (Runtime → Run all)
2. Click the link to open the report
3. Edit and save sections
4. Generate PDF when done

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install -q flask flask-cors weasyprint
print("✅ Installation complete!")

In [ ]:
# Clone the repository (or use uploaded files)
import os
from pathlib import Path

# Check if files exist, if not clone from GitHub
if not os.path.exists('report_template.html'):
    print("📥 Downloading report system files...")
    !git clone https://github.com/M4rceli/report.git
    os.chdir('report')
    print("✅ Files downloaded!")
else:
    print("✅ Files already present!")

# Create necessary directories
Path('saved_sections').mkdir(exist_ok=True)
Path('generated_pdfs').mkdir(exist_ok=True)
print("✅ Directories ready!")

In [ ]:
# Create a simple web server to host the HTML report
from flask import Flask, request, jsonify, send_file, send_from_directory
from flask_cors import CORS
import json
from datetime import datetime
import threading

app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    """Serve the main HTML report"""
    return send_file('report_template.html')

@app.route('/api/save-section', methods=['POST'])
def save_section():
    """Save section data to JSON file"""
    try:
        data = request.json
        section_id = data.get('sectionId')
        timestamp = int(datetime.now().timestamp() * 1000)
        
        filename = f"saved_sections/{section_id}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        return jsonify({
            'success': True, 
            'message': 'Section saved successfully!',
            'filename': filename
        })
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/api/list-sections', methods=['GET'])
def list_sections():
    """List all saved sections"""
    try:
        files = list(Path('saved_sections').glob('*.json'))
        sections = []
        
        for file in files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                sections.append({
                    'filename': file.name,
                    'sectionId': data.get('sectionId'),
                    'author': data.get('author'),
                    'timestamp': data.get('timestamp')
                })
        
        return jsonify({'success': True, 'sections': sections})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/api/load-section/<filename>', methods=['GET'])
def load_section(filename):
    """Load a specific section"""
    try:
        filepath = Path('saved_sections') / filename
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return jsonify({'success': True, 'data': data})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/api/generate-pdf', methods=['POST'])
def generate_pdf():
    """Generate PDF from current report state"""
    try:
        from weasyprint import HTML, CSS
        
        # Load and inject data into HTML
        with open('report_template.html', 'r', encoding='utf-8') as f:
            html_content = f.read()
        
        # Generate PDF
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        pdf_filename = f"generated_pdfs/report_{timestamp}.pdf"
        
        HTML(string=html_content).write_pdf(pdf_filename)
        
        return jsonify({
            'success': True, 
            'message': 'PDF generated successfully!',
            'filename': pdf_filename
        })
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/download/<path:filename>', methods=['GET'])
def download_file(filename):
    """Download generated PDF or JSON file"""
    return send_file(filename, as_attachment=True)

print("✅ Flask server configured!")

In [ ]:
# Start the Flask server in background
import threading

def run_server():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ Server started!")
print("⏳ Waiting for server to initialize...")
import time
time.sleep(3)

In [ ]:
# Create public URL using ngrok or Colab's built-in features
from google.colab.output import eval_js

print("🌐 Creating public URL...")

# Try to use pyngrok for public URL
try:
    !pip install -q pyngrok
    from pyngrok import ngrok
    
    # Set up ngrok
    public_url = ngrok.connect(5000)
    print(f"\n🎉 Your report is ready!")
    print(f"\n📊 Open this URL in your browser:")
    print(f"   {public_url}")
    print(f"\n💡 This URL is accessible from anywhere!")
    print(f"\n⚠️  Note: The URL expires when you stop this notebook.\n")
except Exception as e:
    print("\n⚠️  Ngrok setup failed. Using Colab's local server.")
    print(f"\n📊 Server running at: http://localhost:5000")
    print("\nTo access from outside Colab, you'll need to use port forwarding.\n")

## 📝 How to Use

1. **Open the Report**: Click the URL above
2. **Edit Mode**: Click "🔓 Enable Edit Mode"
3. **Fill Sections**: Add your content
4. **Save**: Click "💾 Save Section" - data saves to Colab!
5. **Continue Later**: Saved data persists in this Colab session
6. **Generate PDF**: Use the cells below

## 💾 Managing Your Data

In [ ]:
# List all saved sections
import json
from pathlib import Path
from datetime import datetime

print("📋 Saved Sections:\n")
print("=" * 60)

json_files = list(Path('saved_sections').glob('*.json'))

if not json_files:
    print("⚠️  No sections saved yet.\n")
else:
    for i, json_file in enumerate(json_files, 1):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"{i}. {json_file.name}")
        print(f"   Section: {data.get('sectionId')}")
        print(f"   Author: {data.get('author', 'Unknown')}")
        print(f"   Date: {data.get('timestamp', 'N/A')}\n")

print("=" * 60)

In [ ]:
# Generate PDF from saved sections
print("📄 Generating PDF...\n")

try:
    from weasyprint import HTML
    import sys
    sys.path.append('.')
    from report_generator import ReportGenerator
    
    generator = ReportGenerator(report_folder='.')
    pdf_path = generator.generate_pdf()
    
    if pdf_path:
        print(f"\n✅ PDF generated successfully!")
        print(f"📁 Location: {pdf_path}")
        print(f"\n💾 Download it using the cell below.")
    else:
        print("\n⚠️  PDF generation failed. Check saved sections.")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Alternative: Use browser print (Ctrl+P) from the report URL")

In [ ]:
# Download generated PDF
from google.colab import files
import os

pdf_files = list(Path('generated_pdfs').glob('*.pdf'))

if pdf_files:
    # Get the most recent PDF
    latest_pdf = max(pdf_files, key=lambda p: p.stat().st_mtime)
    print(f"📥 Downloading: {latest_pdf.name}\n")
    files.download(str(latest_pdf))
    print("✅ Download started!")
else:
    print("⚠️  No PDF files found. Generate one first using the cell above.")

In [ ]:
# Download all saved sections (backup)
from google.colab import files
import shutil

print("📦 Creating backup of all saved sections...\n")

# Create a zip file
shutil.make_archive('saved_sections_backup', 'zip', 'saved_sections')

print("📥 Downloading backup...\n")
files.download('saved_sections_backup.zip')

print("✅ Backup downloaded!")
print("💡 Upload this zip file in future sessions to restore your data.")

In [ ]:
# Upload previously saved sections
from google.colab import files
import shutil
import zipfile

print("📤 Upload your saved sections backup (zip file):\n")

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('saved_sections')
        print("✅ Sections restored!")
    elif filename.endswith('.json'):
        print(f"\n📄 Copying {filename}...")
        shutil.copy(filename, 'saved_sections/')
        print("✅ Section added!")

print("\n🔄 Refresh your report to see the loaded data.")

## 🎯 Tips for Using in Colab

### Saving Your Work
- Data is saved in Colab's runtime, which resets after ~12 hours of inactivity
- **Always download backups** using the backup cell above
- Upload your backup in future sessions to continue

### Sharing with Others
- The ngrok URL works from any device
- Share the URL with colleagues for collaborative editing
- Each person can edit different sections
- Remember: URL expires when you stop the notebook

### Best Practices
1. Save sections frequently
2. Download backups after each session
3. Generate PDF when done
4. Keep the Colab tab open while others are editing

## 🆘 Troubleshooting

### Server not responding?
- Re-run the server cells
- Check if runtime is still active

### Can't save sections?
- Check browser console for errors (F12)
- Try saving directly in Colab using the cells above

### PDF generation fails?
- Use browser print instead (Ctrl+P → Save as PDF)
- Make sure WeasyPrint installed correctly

---

**Made with ❤️ for cloud-based collaborative reporting**